In [1]:
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt


In [6]:
data = pd.read_csv("datasets/online_shoppers.csv", encoding="ISO-8859-1").dropna() # encoding data 

data.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom


# Data Preparation

In [7]:
# Total sales amount
data["Sales"] = data["Quantity"]*data["UnitPrice"]

data.head(0)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Sales


In [12]:
# Simplify Dates

data["InvoiceDate"] = pd.to_datetime(data.InvoiceDate)

# What is the last date when the transactions happened

last_date = data.InvoiceDate.max()


# Date for Recency analysis

recency_analysis_date = last_date + timedelta(days=1)



In [14]:
# Customer Level Aggregation vis the pandas groupby and agg api

df = data.groupby("CustomerID").agg({
    "InvoiceDate" : lambda x: (recency_analysis_date -  x.max()).days,
    "Sales": "sum",
    "InvoiceNo":"count"

})

df.head(10)

,InvoiceDate,Sales,InvoiceNo
CustomerID,,,
12346.0,326,0.00,2
12347.0,2,4310.00,182
12348.0,75,1797.24,31
12349.0,19,1757.55,73
12350.0,310,334.40,17
12352.0,36,1545.41,95
12353.0,204,89.00,4
12354.0,232,1079.40,58
12355.0,214,459.40,13


In [19]:
# Sales per unit invoice number (Basket/ Monetary value)

df["Monetary"] = df.Sales / df.InvoiceNo

df.head(2)

,InvoiceDate,Sales,InvoiceNo,Monetary
CustomerID,,,,
12346.0,326,0.0,2,0.000000
12347.0,2,4310.0,182,23.681319


In [21]:
# Date cleaning and bringing it into RFM form

df=df.drop(columns="Sales")


df.head(3)

,InvoiceDate,InvoiceNo,Monetary
CustomerID,,,
12346.0,326,2,0.000000
12347.0,2,182,23.681319
12348.0,75,31,57.975484


In [22]:

df = df.rename(columns=
          {"InvoiceDate":"Recency",
           "InvoiceNo": "Frequency",
           })

df.head(3)

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,326,2,0.000000
12347.0,2,182,23.681319
12348.0,75,31,57.975484


In [26]:
# Seperating and Ranking the datsets into Quartiles

df["F"] = pd.qcut(
    x=df.Frequency, q=4, labels=range(1,5)
)
df["R"] = pd.qcut(
    x=df.Recency, q=4, labels=range(4,0,-1)  # Lower Recency => last transaction was recent which is a good indicator of loyalty.
)
df["M"] = pd.qcut(
    x=df.Monetary, q=4, labels=range(1,5)
)


df.head()

,Recency,Frequency,Monetary,F,R,M
CustomerID,,,,,,
12346.0,326,2,0.000000,1,1,1
12347.0,2,182,23.681319,4,4,4
12348.0,75,31,57.975484,2,2,4
12349.0,19,73,24.076027,3,3,4
12350.0,310,17,19.670588,1,1,3


In [27]:
# RFM function
# axis 1 is column
df["RFM"] = df[["R","F","M"]].sum(axis=1)

df.head()

,Recency,Frequency,Monetary,F,R,M,RFM
CustomerID,,,,,,,
12346.0,326,2,0.000000,1,1,1,3
12347.0,2,182,23.681319,4,4,4,12
12348.0,75,31,57.975484,2,2,4,8
12349.0,19,73,24.076027,3,3,4,10
12350.0,310,17,19.670588,1,1,3,5


In [28]:
# RFM functions

def RFMFunction(df):
    if df['RFM'] >=11:
        return "Superstar"
    elif df['RFM'] <11 and df['RFM']>=8:
        return "Future Champion"
    elif df['RFM'] <8 and df['RFM']>=6:
        return "High potential"
    else:
        return "Low Relevance"

In [29]:
df["RFM-Score"] = df.apply(RFMFunction,axis=1)

df.head()

,Recency,Frequency,Monetary,F,R,M,RFM,RFM-Score
CustomerID,,,,,,,,
12346.0,326,2,0.000000,1,1,1,3,Low Relevance
12347.0,2,182,23.681319,4,4,4,12,Superstar
12348.0,75,31,57.975484,2,2,4,8,Future Champion
12349.0,19,73,24.076027,3,3,4,10,Future Champion
12350.0,310,17,19.670588,1,1,3,5,Low Relevance


In [30]:
df.value_counts(df["RFM-Score"])

RFM-Score
Future Champion    1768
High potential     1477
Low Relevance       753
Superstar           374
Name: count, dtype: int64

In [32]:
df.groupby("RFM-Score").agg({
    "Recency":["mean"],
    "Frequency":["mean"],
    "Monetary":["mean","count"],ß
})

Recency   Frequency   Monetary      
                       mean        mean       mean count
RFM-Score                                               
Future Champion   32.007919  135.835973  30.169670  1768
High potential   118.708869   39.213947  38.033024  1477
Low Relevance    221.483400   22.811421  -0.586430   753
Superstar          9.973262  244.852941  45.483949   374